In [170]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle
import time

In [152]:
data_dir = './data/'
out_dir = './output/'

# 加载数据

In [175]:
dn = 'dataset_v2.csv'
dataset = pd.read_csv(data_dir + dn)
dataset.head()

,is_intact_0,is_intact_1,is_intact_3,is_intact_2,is_intact_5,is_intact_6,is_intact_4,is_intact_7,is_intact_8,classify_id_4,...,tags_intersect_ratio,key_word_intersect_count,key_word_intersect_ratio,cid_,is_intact_,series_id,duration,title_length,upgc_flag,classify_id_
0,5,0,0,0,0,0,0,0,0,0,...,0.0,8,0.470588,4205,0,103767,2972,114,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0.0,8,0.470588,4205,0,103767,2972,114,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0.0,9,0.600000,1384,0,0,75,59,1,12
3,5,0,0,0,0,0,0,0,0,0,...,0.0,8,0.421053,772,0,19033,44,63,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0.0,2,0.086957,2852,1,106327,30,51,0,0


In [195]:
test_dn = 'test_dataset_v2.csv'
test_dataset = pd.read_csv(data_dir + test_dn)
test_dataset.head()

,is_intact_0,is_intact_1,is_intact_3,is_intact_2,is_intact_5,is_intact_6,is_intact_4,is_intact_7,is_intact_8,classify_id_4,...,tags_intersect_ratio,key_word_intersect_count,key_word_intersect_ratio,cid_,is_intact_,series_id,duration,title_length,upgc_flag,classify_id_
0,6,0,0,0,0,0,0,0,0,0,...,0.0,14,1.000000,6342,0,0,2649,24,0,1
1,6,0,0,0,0,0,0,0,0,0,...,0.0,14,1.000000,4233,0,0,162,29,0,1
2,6,0,0,0,0,0,0,0,0,0,...,0.0,3,0.120000,2852,1,106327,30,51,0,0
3,6,0,0,0,0,0,0,0,0,0,...,0.0,1,0.033333,3516,1,105454,20,63,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0.0,3,0.111111,2169,0,103244,1407,29,0,4


# 训练集和验证集划分

目前这个划分是比较随机的，按理来说应该按照用户来分。

In [43]:
val_ratio = 0.2

In [176]:
y = dataset['label']
did = dataset['did']
vid = dataset['vid']
dataset.drop(['did', 'vid', 'label'], axis=1, inplace=True)
x = dataset
x.shape

(1879999, 58)

In [196]:
test_did = test_dataset['did']
test_vid = test_dataset['vid']
test_dataset.drop(['did', 'vid', 'label'], axis=1, inplace=True)
test_x = test_dataset
test_x.shape

(3417591, 58)

In [197]:
did.nunique() == test_did.nunique()

True

## 随机划分

In [44]:
idxs     = dataset.index.values
pos_idxs = idxs[y == 1]
neg_idxs = idxs[y == 0]

In [45]:
pos_val_num = int(len(pos_idxs)*val_ratio)
neg_val_num = int(len(neg_idxs)*val_ratio)
np.random.shuffle(pos_idxs)
np.random.shuffle(neg_idxs)

In [46]:
pos_val_idxs, pos_tra_idxs = pos_idxs[:pos_val_num], pos_idxs[pos_val_num:]
neg_val_idxs, neg_tra_idxs = neg_idxs[:neg_val_num], neg_idxs[neg_val_num:]
val_idxs = np.concatenate([pos_val_idxs, neg_val_idxs])
tra_idxs = np.concatenate([pos_tra_idxs, neg_tra_idxs])

In [47]:
tra_idxs.shape[0] + val_idxs.shape[0] == idxs.shape[0]

True

In [56]:
tra_x, val_x = x.iloc[tra_idxs], x.iloc[val_idxs]
tra_y, val_y = y.iloc[tra_idxs], y.iloc[val_idxs]

In [79]:
did.iloc[val_idxs]

1043339     94849
1191619    108329
395670      35970
207218      18838
515471      46861
            ...  
928013      84364
878143      79831
141709      12882
1714312    155846
1671180    151925
Name: did, Length: 375999, dtype: int64

## 按照用户来划分训练集和验证集

In [177]:
all_did = did.unique()

In [178]:
val_did_num = int(len(all_did)*val_ratio)
np.random.shuffle(all_did)
val_did, tra_did = all_did[:val_did_num], all_did[val_did_num:]

In [179]:
idxs = x.index.values
tra_idxs, val_idxs = idxs[did.isin(tra_idxs)], idxs[did.isin(val_idxs)]
tra_idxs.shape[0] + val_idxs.shape[0] == len(idxs)

True

In [180]:
tra_x, val_x = x.iloc[tra_idxs], x.iloc[val_idxs]
tra_y, val_y = y.iloc[tra_idxs], y.iloc[val_idxs]

# 模型构建与训练

In [77]:
# def decorator(did, vid):
def mrr(y_true, y_pred, did):
    df = pd.DataFrame({'true': y_true, 'pred': y_pred, 'did': did})
    g = df.groupby('did')
    
    s = 0
    for k, v in g:
        v = v.sort_values(by='pred', ascending=False)
        v = v.iloc[:6]['true'].tolist()
        if 1 in v:
            s += 1 / (v.index(1) + 1)
            
    return s / len(g)
    # return mrr

In [181]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'learning_rate': 0.01,
    'num_tree': 300,
    'num_leaves': '64',
    'metric': 'auc',
}

In [182]:
cat_feats = ['cid_', 'is_intact_', 'upgc_flag', 'classify_id_']
# fns = dataset.columns.tolist()
# fns.remove('label')
# fns.remove('did')
# fns.remove('vid')
tra_dataset = lgb.Dataset(tra_x, label=tra_y, categorical_feature=cat_feats)
val_dataset = lgb.Dataset(val_x, label=val_y, categorical_feature=cat_feats)
tes_dataset = lgb.Dataset(test_x, label=None, categorical_feature=cat_feats)

In [123]:
tra_x.shape, val_x.shape

((1505988, 58), (374011, 58))

In [183]:
model = lgb.train(params, 
                  # feval=mrr,
                  train_set=tra_dataset, valid_sets=[tra_dataset, val_dataset], valid_names=['train', 'val'], 
                  verbose_eval=1)

/home/gzy/.conda/envs/intern/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_tree` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/gzy/.conda/envs/intern/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/gzy/.conda/envs/intern/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 142958, number of negative: 1429580
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045482 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8450
[LightGBM] [Info] Number of data points in the train set: 1572538, number of used features: 58


/home/gzy/.conda/envs/intern/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/gzy/.conda/envs/intern/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.090909 -> initscore=-2.302585
[LightGBM] [Info] Start training from score -2.302585
[1]	train's auc: 0.700789	val's auc: 0.697523
[2]	train's auc: 0.701843	val's auc: 0.698716
[3]	train's auc: 0.702926	val's auc: 0.699875
[4]	train's auc: 0.703157	val's auc: 0.700087
[5]	train's auc: 0.707491	val's auc: 0.704622
[6]	train's auc: 0.707892	val's auc: 0.704723
[7]	train's auc: 0.708308	val's auc: 0.704938
[8]	train's auc: 0.708374	val's auc: 0.70488
[9]	train's auc: 0.711374	val's auc: 0.707867
[10]	train's auc: 0.71248	val's auc: 0.708954
[11]	train's auc: 0.712486	val's auc: 0.709009
[12]	train's auc: 0.71314	val's auc: 0.709526
[13]	train's auc: 0.713279	val's auc: 0.709697
[14]	train's auc: 0.713851	val's auc: 0.710313
[15]	train's auc: 0.714707	val's auc: 0.711075
[16]	train's auc: 0.714909	val's auc: 0.711183
[17]	train's auc: 0.71597	val's auc: 0.712284
[18]	train's auc: 0.716586	val's auc: 0.712877
[19]	train's auc: 0.716676	val's 

In [184]:
y_pred = model.predict(val_x)
val_did = did.iloc[val_idxs]
mrr(val_y, y_pred, val_did)

0.5469547899300263

# 推理

In [194]:
from tools import checkSubmit, saveSubmit

ImportError: cannot import name 'saveSubmit' from 'tools' (/home/gzy/jupyter-lab/mgtv-2022/tools.py)

In [136]:
def generate_submit(did, vid, score):
    df = pd.DataFrame({'did': did, 'vid': vid, 'score': score})
    g = df.groupby('did')
    did_col, vid_col, rank = [], [], [1, 2, 3, 4, 5, 6]*len(g)
    
    for k, v in g:
        did_col.extend([k] * 6)
        v = v.sort_values(by='score', ascending=False).iloc[:6]
        vid_col.extend(v['vid'].to_list())
    
    df = pd.DataFrame({'did': did_col, 'vid': vid_col, 'rank': rank})
    
    return df    

In [142]:
with open(data_dir + 'index_map.pkl', 'rb') as f:
    index_map  = pickle.load(f)
    dindex_map = index_map['did']
    vindex_map = index_map['vid']
    
inv_dindex_map = {v:k for k, v in dindex_map.items()}
inv_vindex_map = {v:k for k, v in vindex_map.items()}

In [198]:
y_pred = model.predict(test_x)

In [199]:
submit = generate_submit(test_did, test_vid, y_pred)
submit

,did,vid,rank
0,0,118609,1
1,0,55520,2
2,0,43662,3
3,0,7358,4
4,0,94570,5
...,...,...,...
1025449,170908,3906,2
1025450,170908,61519,3
1025451,170908,143981,4
1025452,170908,35500,5


In [200]:
submit['did'] = submit['did'].apply(lambda x: inv_dindex_map[x])
submit['vid'] = submit['vid'].apply(lambda x: inv_vindex_map[x])

In [193]:
submit.iloc[6:12]

,did,vid,rank
6,00012884369ff7fd10208962e4a72666,42620daa94e5039db423be8a1af47e10,1
7,00012884369ff7fd10208962e4a72666,db978f94b983a6ed3021e1e927ee09ef,2
8,00012884369ff7fd10208962e4a72666,2b9ea277f728d4ed80ddc89986d105a3,3
9,00012884369ff7fd10208962e4a72666,28ec1245290111350d15773fa5f98d37,4
10,00012884369ff7fd10208962e4a72666,416717dc56fb7eabecf419becf1a1cff,5
11,00012884369ff7fd10208962e4a72666,5ebeac396a6b2b150c162c5c899fe20f,6


In [201]:
fn = time.strftime('%Y%m%d-%H:%M:%S')
fn += '.v1.v3.lgb.csv'
fn

'20220717-12:21:41.v1.v3.lgb.csv'

In [202]:
checkSubmit(submit)

Congratulations, submit file passes the checks!


In [203]:
submit.to_csv(out_dir + fn, index=False)
print('二阶的关联推荐结果保存至 %s%s' % (out_dir, fn))

二阶的关联推荐结果保存至 ./output/20220717-12:21:41.v1.v3.lgb.csv


In [158]:
test_dataset[['did', 'vid']].iloc[:10]

,did,vid
0,0,55520
1,0,118609
2,0,129271
3,0,93812
4,0,7358
5,0,43662
6,0,128217
7,0,58921
8,0,25648
9,0,140248


In [166]:
tmpg = test_dataset[['did', 'vid']].groupby('did')

submit = pd.DataFrame(columns=['did', 'vid', 'rank'])
rank = [1, 2, 3, 4, 5, 6]
did_ = []
vid_ = []
rank = [1, 2, 3, 4, 5, 6] * len(tmpg)
for k, v in tmpg:
    did_.extend([k]*6)
    vid_.extend(v['vid'].to_list()[:6])

submit['did'] = did_
submit['vid'] = vid_
submit['rank'] = rank

In [167]:
submit['did'] = submit['did'].apply(lambda x: inv_dindex_map[x])
submit['vid'] = submit['vid'].apply(lambda x: inv_vindex_map[x])

In [172]:
def saveSubmit(df, suffix='', out_dir='./output/'):
    checkSubmit(df)
    fn = time.strftime('%Y%m%d-%H:%M:%S')
    fn += f'.{suffix}.csv'
    
    submit.to_csv(os.path.join(out_dir, fn), index=False)
    print('二阶的关联推荐结果保存至 %s%s' % (out_dir, fn))

In [174]:
saveSubmit(submit, suffix='2nd-associated.v1')

Congratulations, submit file passes the checks!
二阶的关联推荐结果保存至 ./output/20220716-23:55:06.2nd-associated.v1.csv
